In [1]:
import pickle
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import punkt
from nltk.corpus.reader import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
path_models = r"C:\ICE-1\ICE-1\04 Model Training\Models/"

# SVM
path_svm = path_models + 'best_svc.pickle'
with open(path_svm, 'rb') as data:
    svc_model = pickle.load(data)

In [3]:
path_tfidf = r"C:\ICE-1\ICE-1\03 Feature Engineering\Pickles\tfidf.pickle"
with open(path_tfidf, 'rb') as data:
    tfidf = pickle.load(data)

In [4]:
category_codes = {
    'Autonomous Cars': 0,
    'Bill Resource': 1,
    'Companies Working on Autonomous Cars': 2
}


In [5]:
punctuation_signs = list("?:!.,;")
stop_words = list(stopwords.words('english'))

def create_features_from_text(text):
    
    # Dataframe creation
    lemmatized_text_list = []
    df = pd.DataFrame(columns=['Article Content'])
    df.loc[0] = text
    df['Content_Parsed_1'] = df['Article Content'].str.replace("\r", " ")
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')
    df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()
    df['Content_Parsed_3'] = df['Content_Parsed_2']
    for punct_sign in punctuation_signs:
        df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
    df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized_list = []
    text = df.loc[0]['Content_Parsed_4']
    text_words = text.split(" ")
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
    lemmatized_text = " ".join(lemmatized_list)    
    lemmatized_text_list.append(lemmatized_text)
    df['Content_Parsed_5'] = lemmatized_text_list
    df['Content_Parsed_6'] = df['Content_Parsed_5']
    for stop_word in stop_words:
        regex_stopword = r"\b" + stop_word + r"\b"
        df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')
    df = df[['Content_Parsed_6']]
    df = df.rename(columns={"Content_Parsed_6": "Content_Parsed"})
    #df = df.drop("Content_Parsed_6", axis = 1)
    
    # TF-IDF
    features = tfidf.transform(df).toarray()
    
    return features

In [6]:
def get_category_name(category_id):
    for category, id_ in category_codes.items():    
        if id_ == category_id:
            return category

In [7]:
def predict_from_text(text):
    
    # Predict using the input model
    prediction_svc = svc_model.predict(create_features_from_text(text))[0]
    prediction_svc_proba = svc_model.predict_proba(create_features_from_text(text))[0]
    
    # Return result
    category_svc = get_category_name(prediction_svc)
    
    print("The predicted category using the SVM model is %s." %(category_svc) )
    print("The conditional probability is: %a" %(prediction_svc_proba.max()*100))

In [8]:
text = """The term ‘autopilot’ has been used for decades within the aircraft industry, enabling pilots to reduce their cognitive load by allowing an aircraft to practically fly itself during “cruising” portions of the trip. The same is now happening as cars with autopilot are rapidly turning into more autonomous, self-driving vehicles, allowing drivers to let the cars drive themselves on certain portions of the trip, like freeways.

Tesla most famously branded its self-driving features as “Autopilot”, but many other automakers are now including similar self-driving and autonomous vehicle features as part of their driving assistance option packages.

This article provides a breakdown of cars with autopilot and who offers what, but first, we’ll go through a bit of background and explanation of terms.
Autopilot is a generic term used in multiple industries, particularly aircraft, and generally means that something can guide itself without human intervention. In relation to cars in 2021, most mainstream carmakers are focused on, Level 2 autonomy. This level allows the vehicle to take over most steering, acceleration and braking functions, but still requires that the driver remain fully attentive to the driving situation and be able to intervene at any moment. It is not driverless, fully autonomous driving, like robotaxis from Waymo or Cruise (that are now testing in California).

That means, today, autopilot really means ‘assisted driving’ and not ‘self-driving’ since the driver still has to be alert and attentive at all times. It won’t be until Level 4 or Level 5 fully autonomous cars hit the roads that the true promise of full self-driving will be a reality. Currently, that’s not expected to happen until 2022 (although the team at Tesla is pushing hard to do so as soon as possible, as announced at Tesla Autonomy Day, since all new cars sold with Hardware 3 are much more powerful).

Learn more about autonomous Full Self-Driving capabilities that Tesla currently includes today.
"""

In [9]:
predict_from_text(text)

<ipython-input-5-13c7c93b1644>:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')


The predicted category using the SVM model is Bill Resource.
The conditional probability is: 89.781405633369


<ipython-input-5-13c7c93b1644>:31: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')
<ipython-input-5-13c7c93b1644>:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
<ipython-input-5-13c7c93b1644>:31: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')


In [10]:
text = """
28. "Electric bicycle" means a bicycle or tricycle that is equipped with fully operable pedals and an electric motor of less than seven hundred fifty watts and that meets the requirements of one of the following classes:

(a) "Class 1 electric bicycle" means a bicycle or tricycle that is equipped with an electric motor that provides assistance only when the rider is pedaling and that ceases to provide assistance when the bicycle or tricycle reaches the speed of twenty miles per hour.

(b) "Class 2 electric bicycle" means a bicycle or tricycle that is equipped with an electric motor that may be used exclusively to propel the bicycle or tricycle and that is not capable of providing assistance when the bicycle or tricycle reaches the speed of twenty miles per hour.

(c) "Class 3 electric bicycle" means a bicycle or tricycle that is equipped with an electric motor that provides assistance only when the rider is pedaling and that ceases to provide assistance when the bicycle or tricycle reaches the speed of twenty-eight miles per hour.

26. 29. "Electric miniature scooter" means a device that:

(a) Weighs less than thirty pounds.

(b) Has two or three wheels.

(c) Has handlebars.

(d) Has a floorboard on which a person may stand while riding.

(e) Is powered by an electric motor or human power, or both.

(f) Has a maximum speed that does not exceed ten miles per hour, with or without human propulsion, on a paved level surface.

27. 30. "Electric personal assistive mobility device" means a self-balancing device with one wheel or two nontandem wheels and an electric propulsion system that limits the maximum speed of the device to fifteen miles per hour or less and that is designed to transport only one person.
"""

In [11]:
predict_from_text(text)

The predicted category using the SVM model is Bill Resource.
The conditional probability is: 89.781405633369


<ipython-input-5-13c7c93b1644>:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
<ipython-input-5-13c7c93b1644>:31: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')
<ipython-input-5-13c7c93b1644>:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
<ipython-input-5-13c7c93b1644>:31: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['C

In [12]:
text = """
The Rivian and Aurora investments aren’t the only autonomous technology play that Amazon is pursuing. In January 2019, the company introduced the Amazon Scout, a 6-wheeled electric-powered delivery robot. These robots are currently making deliveries in a Washington neighborhood and in Southern California’s Irvine area during daylight hours, Monday through Friday, under the supervision of a human associate. In July 2020, the project expanded its geographical coverage. Amazon’s delivery robots will now serve select customers in Franklin, Tennessee and Atlanta, Georgia.

Additionally, in June 2020, Amazon announced plans to acquire Zoox, a startup developing autonomous driving tech geared toward ride-hailing customers. This $1.2B deal provides the e-commerce giant with access to software, AI, and a full-stack self-driving solution. Backed by Amazon, Zoox is now better equipped to take on its primary rival, Waymo, an Alphabet subsidiary. In December 2020, Zoox unveiled its fully autonomous, electric robotaxi, which can seat 4 passengers and reach speeds of up to 75 mph. The vehicle is currently in testing.

For Amazon, these developments are years in the making.

The company initially announced that it would be getting involved in autonomous vehicles at CES 2018 through a partnership with Toyota. The demo vehicle, known as the e-Palette, was designed as a multi-function, autonomous minivan to move goods, people, or even a mobile office. The plan was to debut up to 20 of its e-Palette vehicles at the 2020 Summer Olympics and Paralympic Games in Tokyo to transport athletes. But the Covid-19 pandemic forced Japanese organizers to postpone these sports events until July 2021.

In 2015, Amazon explored a trial with DHL and Audi that involved delivering customers’ parcels to the trunks of their automobiles.

In the same year, Amazon filed for a patent for a system that helps autonomous cars navigate roadways, especially complex, reversible lanes, and even pick which lane to use depending on current traffic estimates — an early indicator of the company’s ambitions in this space and AVs’ importance to Amazon when it comes to lowering delivery costs. The company was granted the patent in early 2017.

In April 2017, the Wall Street Journal reported that Amazon had built a team more than a year prior devoted to focusing on driverless vehicle technology
"""

In [13]:
predict_from_text(text)

The predicted category using the SVM model is Bill Resource.
The conditional probability is: 89.781405633369


<ipython-input-5-13c7c93b1644>:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
<ipython-input-5-13c7c93b1644>:31: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')
<ipython-input-5-13c7c93b1644>:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
<ipython-input-5-13c7c93b1644>:31: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['C

In [14]:
# Weather

text = """
A polar air mass that entered the Iberian peninsula on Wednesday has already caused sharp drops in temperature, but the worst is yet to come, said the national weather service Aemet.

“An episode of intense cold” is forecast for Friday, when the mercury will continue to plummet across much of Spain. Thirty provinces are on yellow alert, the lowest on a three-color scale, except for Guadalajara, where there is an orange advisory in place. Lows of -10 ºC are expected in Parameras de Molina.

Elsewhere, weather stations have recorded -8.2ºC in La Molina (Girona), at an elevation of 1,700 meters, and -6.8ºC in Puerto de Navacerrada (Madrid).

 Almería has rolled out vehicles to deal with wintry road conditions.
Almería has rolled out vehicles to deal with wintry road conditions. DIPUTACIÓN DE ALMERÍA EUROPA PRESS
Aemet spokesman Rubén del Campo said that the cold spell is not out of the ordinary for a month of January, and noted that “we are emerging from a warm December when temperatures were 1.2ºC above average.”

Temperatures have already dipped between six and eight degrees in a matter of hours in some parts of Spain, said Del Campo. And the northerly wind will increase the feeling of cold.

Temperatures on Friday and Saturday will be “very cold, with lows of five to 10 degrees below average in many parts of northern and eastern Spain, and in the Balearics,” he added.

No snow
However, little to no snow is expected “not for lack of cold, but for lack of precipitation, since the air mass is very dry.”

Alerts are in place in Almería, Granada, Jaén, Aragón, Cantabria, Castilla-La Mancha, Castilla y León except for Salamanca and Valladolid, Catalonia save for Tarragona, Madrid, Navarre, the Valencia region, the Balearic Islands, La Rioja, Asturias and Murcia.

On Saturday, the orange warnings will extend to Córdoba, Salamanca, Valladolid, Galicia and Lleida, where lows of -10 ºC are expected in the Vall d’Aran.


"""

predict_from_text(text)

The predicted category using the SVM model is Bill Resource.
The conditional probability is: 89.781405633369


<ipython-input-5-13c7c93b1644>:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
<ipython-input-5-13c7c93b1644>:31: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')
<ipython-input-5-13c7c93b1644>:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
<ipython-input-5-13c7c93b1644>:31: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['C

In [15]:
# Health

text = """
The obesity epidemic has been on the rise for years, with cases nearly tripling since 1975, according to the World Health Organization (WHO). And the outlook for the future is equally bleak – and that includes Spain, a country that is famed for the positive effects of its supposedly Mediterranean diet.

An investigation by the Mar de Barcelona hospital has found that 80% of men and 55% of women will be overweight by 2030. If the current trend continues, the researchers write, within 11 years more than 27 million people in Spain will be overweight.

Being overweight can mean a higher risk of suffering a number of diseases, including diabetes, hypertension, stroke and cancer

The study, published in the Spanish Cardiology Magazine, points out that this epidemic will mean costs of €3 billion for the country’s health system.

The issue, the experts state, is not an esthetic one, but rather a question of health. Being overweight can mean a higher risk of suffering a number of diseases, including diabetes, hypertension, stroke, cancer and other cardiovascular conditions.

Researchers at the Barcelona hospital revised all of the scientific literature published in Spain on epidemiology and the prevalence of obesity and excess weight between 1987 and 2014 in order to come up with an accurate picture of the state of the country’s residents. “We found the trends and we cross-referenced them with data covering the general population from the National Statistics Institute in order to estimate the figures of obesity and excess weight,” explains Álvaro Hernáez, the chief researcher on the project. The results revealed an “alarming future.”

There are currently 25 million people with excess weight, three million more than a decade ago

DR ALBERT GODAY, AUTHOR OF THE STUDY

“There are currently 25 million people with excess weight, three million more than a decade ago,” explains Dr Albert Goday, another author of the study. “In a conservative scenario, if this trend continues, there will be another three million more – 16% more cases – by 2030.”

“In men, excess weight is more usual up to the age of 50,” explains Goday. “From 50 onward, obesity rises among women. These are issues that are intrinsically linked to hormonal metabolism. From a certain age, it is harder for women to control their weight.”

The experts argue that any weight loss, no matter how small, reduces the risk of contracting one of the associated illnesses. “We aren’t going to find the key to this or a vaccination overnight,” Goday adds. “But any minor intervention will help to reduce the risk. You might think that you have made a huge effort and have only lost four kilos when you need to lose many more, but you must bear in mind that those four kilos are already reducing the risk.”

"""

predict_from_text(text)


The predicted category using the SVM model is Bill Resource.
The conditional probability is: 89.781405633369


<ipython-input-5-13c7c93b1644>:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
<ipython-input-5-13c7c93b1644>:31: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')
<ipython-input-5-13c7c93b1644>:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
<ipython-input-5-13c7c93b1644>:31: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['C